In [1]:
print("sairam")

sairam


In [58]:
import tensorflow as tf
print(tf.__version__)

from tensorflow import keras
from tensorflow.keras.layers import Dense, Activation, Dropout, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.models import Sequential

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
import numpy as np

2.3.0


In [11]:
train_datagen = ImageDataGenerator(rescale=1./255, horizontal_flip=True)
test_datagen  = ImageDataGenerator(rescale=1./255, horizontal_flip=True)

In [21]:
training_set = train_datagen.flow_from_directory("E:\OneNeuron\Computer Vision_Sudhanshu\Image Classification\data\Train",
                                                target_size=(64, 64),
                                                batch_size=32
                                                )

test_set = test_datagen.flow_from_directory("E:\OneNeuron\Computer Vision_Sudhanshu\Image Classification\data\Valid",
                                                target_size=(64, 64),
                                                batch_size=32
                                                )

Found 487 images belonging to 6 classes.
Found 30 images belonging to 6 classes.


In [32]:
classifier = Sequential()

#Trying to follow the sequence described in https://poloclub.github.io/cnn-explainer/
classifier.add(Conv2D(filters=32, kernel_size=(3,3), strides=(1,1), activation='relu', input_shape=(64,64,3)))
classifier.add(Conv2D(filters=32, kernel_size=(3,3), strides=(1,1), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))

classifier.add(Conv2D(filters=32, kernel_size=(3,3), strides=(1,1), activation='relu'))
classifier.add(Conv2D(filters=32, kernel_size=(3,3), strides=(1,1), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))

classifier.add(Flatten())

classifier.add(Dense(units=128, activation='relu'))
classifier.add(Dense(units=6, activation='softmax'))
               

In [33]:
classifier.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_13 (Conv2D)           (None, 62, 62, 32)        896       
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 60, 60, 32)        9248      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 30, 30, 32)        0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 28, 28, 32)        9248      
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 26, 26, 32)        9248      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 5408)             

In [34]:
classifier.compile(metrics=['accuracy'], loss='categorical_crossentropy', optimizer='adam')

In [35]:
model = classifier.fit_generator(training_set, epochs=10, validation_data=test_set, steps_per_epoch=16)

Epoch 1/10
16/16 [==============================] - 2s 110ms/step - loss: 1.5588 - accuracy: 0.4723 - val_loss: 2.0318 - val_accuracy: 0.1667
Epoch 2/10
16/16 [==============================] - 2s 101ms/step - loss: 1.2827 - accuracy: 0.5298 - val_loss: 1.4667 - val_accuracy: 0.5000
Epoch 3/10
16/16 [==============================] - 2s 102ms/step - loss: 0.6191 - accuracy: 0.7926 - val_loss: 0.8423 - val_accuracy: 0.6333
Epoch 4/10
16/16 [==============================] - 2s 103ms/step - loss: 0.3097 - accuracy: 0.9035 - val_loss: 0.6384 - val_accuracy: 0.8000
Epoch 5/10
16/16 [==============================] - 2s 130ms/step - loss: 0.2088 - accuracy: 0.9199 - val_loss: 0.5783 - val_accuracy: 0.7667
Epoch 6/10
16/16 [==============================] - 2s 134ms/step - loss: 0.1493 - accuracy: 0.9384 - val_loss: 0.8783 - val_accuracy: 0.8000
Epoch 7/10
16/16 [==============================] - 2s 136ms/step - loss: 0.1112 - accuracy: 0.9548 - val_loss: 0.5589 - val_accuracy: 0.9000
Epoch 

In [37]:
classifier.save('chess_piece_classifier.h5')

<b> Inference

In [87]:
test_img = "E:\\OneNeuron\\Computer Vision_Sudhanshu\\Image Classification\\data\\Valid\\king\\2.png"

test_image = image.load_img(test_img, target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis=0)
y_hat = classifier.predict(test_image)
y_hat_idx = np.argmax(y_hat)

print(y_hat)

class_idx = training_set.class_indices

[{v, k} for k,v in class_idx.items() if v == y_hat_idx][0]

[[0. 1. 0. 0. 0. 0.]]


{1, 'king'}

In [2]:
import base64

In [3]:
encoded_text = 'iVBORw0KGgoAAAANSUhEUgAAAFUAAABVCAIAAAC3lz8NAAAgAElEQVR4AYzBCZSd5WEm6Pf9/uXudau0i10IgSwJMJjVGIwxYBsM3nDseGfxnqT7pKe3SefMnOnTnen0nD7dp53EcdyJYyc2MTbGLEIgJLRLCCGV2CSVJIRUkkqlparu+q/f9869JYHAdk/P8/CRX/+6NX5s3XPPSgSEPkoOAEnDPkHTHEAAxlACSc8YGjqJIEhJIHEaiR4JfRIACU44S+gRAEE4g4QBSYAgQZwl9BGQ0CNAIiHJ0ABQn5MTCCeBAOF5viSQINFD9MhBcsZ4kOOjjz8+dWxs9Yrl1rkgCDzPl5wxHgUQBAQQMMZgmvAOEkkYEuyBIc4gzhIASZAg4TdIgCCARA/7QIL4nQQBoAABEgBNA0ASAAGhTwQgAMZ4DiIJQAAISc7aOE0g8dEnnpgcO7p6+VPG83zfz23urAVBECQggjiNIAlAAiAIPSRBAqAxAEH8JgEQ1AeIwm8RBJAgQIAEibOIPqFHAAShT4L6cJoA9qCHRI+IM0iQIAE4gRTBIAgAxHHER596auLI4eeXLw8Koc3zzOZEHwGQmEZCAiAQECAIIMBpMAYEQYAgzhDeIqgHkDCNBA1JQ9CwB6cJECTBQYIgCH3ENAnTJFFCj/ogSEIPcRoNiWmEQBA07BEoiIBAz5gwLKRpwl899dTEkcPPP73cC4I0TQH1sAcEQUAAJBB9Qo+cQPA0Y0CCIAgQxBnCGeqDhB7SGBrj+Z7n+V6f8YwhSEhOLrc2t9bm1jrrJEg4TQCEPkGABPUAktAjCEIfiR6SAGgogCRIUKQRBKJHQiEskuBjy5efOjy65pmnPc9P00SAIII9gkhCEGRIJ1E9kEDCGMNpIkESRA9xhjBNECRBYo8xge8HYU8QhmEQBJ7vG2NISrLWZnmepkmaplma5XlunZWEHgl9ggAJ6gEgCdOEaRL6CEMKMIYARAAkAYKk0CcgDMLAD/jrp58+NXpozbMrPM9L0kQSSYBCjwCSgCAAEiShj6Sh4TQYoo8gzhL6JACSA+gZzw/8YqFYLBVLpVKhUPCDwPM8YwwNJVnnsixLkySKoziKkiTJsszmVnI4TYIE9QCS0CO8RTiD6CEJECRFnMFpBhIkFcJCEAR8/JkVpw6NPv/Mcs/zkjQBQFI9oCT2QYIkSJhG8AzDHpAAQZwlnKE+AIYmCIJisVipVMqVSqlUDMIQpLW5tY6Gnud7vgcgy7I4jrqdTrfTiaIoyzJnrSRIkCAJgHoACNOE3yLQACAJkCD6SBDGGPVAYVAIg4BPrnz25KFDz69YboxJkoSkAEHoIwkSzkESJPTRECANDUmQINFDnCX0SYIgkPQ9v1AsVKvVWm2gXCn7vt/pdo+Nj48fP9HpdklWKuU5c2bPmzevWinnuY263Var1Wm34jjO0kzOQQLUA/UAEKYJv0XoIQGwzwCGkkASAAmDnsAvhEHAp1Y+e3L00PMrlpNekiaEhD4BxhhJAJxTDwQCfJshadBDgngXAQKgHgCeMWEYViqV2kC9Wqt6nnf85Mn9B948duJk5pwXhjmQRHHBN+fNnbNwwUVzZ88m2W61mo2pTrudJInNc0jQaQCEaRJA/CaBhASwzxAwBCSAhEBjCND3wzAM+NRzKydGD616erkxJstSOQkSSAAESOckJ0AACAKgoaEB+0CC+E0CJAGQCPi+XyyVarWB+uBgGAbHjh9/ZdfuVpJedOmlly5dOnPe/Ci3bxwe3b9716kDB2aEwVWXLzv33PlJFDemplrNRtTpZlkq5zQNgvAOBITfREAAwT6AhgY9kmgIgKTvh2EQcvlzKydHR59bsdwzJs1SOTk5gD2CADinHvQIJEEYGhr2iCQI4p0k9EmQIJEMw7BSrdbrgwMDA+1ud/PWrY0kvfXuuz9450fOvfCCsFDs5PZwo7Fv9NDwmrWvr1p5bq1643XXFIKg2Wg0pqY6rVaSxM5aSRCE3yT8JgIknWRIEOwxxGkECZC+H4RByOXPrZwcPbxqxXLP85I0kZyzAkESgADnHNQD9gGgoaEhSJDoIc4ShD6qx0EwNIVioVarDw4NlquV/W+8uX7LlmtuvfVr3/3u/PPPF5A6NbP8VJI0c3t87Oj6h382umXzjVddufCii9qt1tTERKvZiKPIWisJv4vwWwRjKAkADWlIEJQEECRhGPphEARc/tzKydHDq59Z7vl+kibOWkkASagPzqkHAPvAPkMSJEj0EGcJfRLUB8AYUyyV6gP1+uCQHwbbd+48cvLU577xjTvvuZeGkXUnkvRkkqQSpCzLXlm3dv2P//7C+sDN778hieLJiVONqalut2tzCwjThP8FCcZAQg/7QM8AghNpRIEsBGEQhnxq5crJw6Orn3na9/00S51zkuAEQoCck5N1MoYEzzAEDQiAIN5F6JOgPgCe8YqlYr0+ODg05MgtL77ISvUrf/CHV113XTfNxuNkPElSOQ8whKR9wzuf/evvV6LORz58m83ziYmJxtRk1O3aPMc04f8nAsI0Y2iMEYUeAeyB7weFsMDlz62cGB1d/czTvu9nWeqcAPUA0DRnnXMwhiRIgoYkSJAg3kXokyBBPegxnlcqFQcG64NDQ4DZ8uI2VSpf+YM/vOq6a09048NR1LHWkIAMBHHf8I5nf/CDStS547YP2SybnDg1NTUZdSNrLd5B+F8RSAggQNLzDHoISeghfD8sFEI+veq5U4dGVz+z3Pe8NE8ByskYOudknYNk1QcYQ9KQhCFIgCCmEX2CAAk96oMAyBhTLJUGBuuDQ0NBWNi+c+fYxOQXvvXtW++662g3OhrHuWTQZwCb5y+vW/f83/3d+QPVW268MYq6UxOnphqNqBs5Z/EW4XcQziDOkEBCgjEg6RkjEs6BBOH7XiEscPmq5yZGR1evWO55Xpql6BGMIQDnnM2tJAA0lANpaAASJEAQbyF6JPRIcBIEoYeeKRQKtdrA0IyhSq02snffxq0v3vGZz3zuG99oB+GxOHYAAQKUus3Wpsce2/nUE+97z2WLFy1qt1qTExPNRiNJEucspgkgIaFHAojfTQAxTYY0xrAHfU4C4IdB6Ad8etVzJw+NPv/MU77nJ1kqyRhD9ElyfYIkgGAPDEGih0QPSfA0CASgMyA5iGQQhtVqtT40VK/Xx48fX/HcqkXvveqhf/G/lc49byzqZhJBypEcP3ho9Y9/3BzZdcsN1w8ODDQajanJyXa7naWpkwNpSJAAJEEQBEDTIAjCW4RpAgkShqQxRJ8kAH4YhF7Ap1evOnnw0PPPPOV7fpZn1lmCNIQAwlkHQAIggAJpiB7S0NCc4Xme4TQQ6oHknJy1VpLn+cVyaaCnXs9svnrNugTmq//sjy59/03HkyRxDoCcSI68tG3VD384JHvLTTfKqTHVaDSmoiiyzhmSpoc0BtMkSM45OWut65OzmgZAAonTBHmmDxKNkXMEvcAEfsinV686dWh09dNPer6fZakkkiB6nHMAJAIgKQEEe0yP53ue5/tBEHi+73u+12MMegTJSc5aZ521zkryPL9ULtcGaoVi8dXXdu14fddNH7/nts99Pq+UM+ck9ORJuvXJJ1989JHLL7542ZLFcRy3Gs1Op51lGY3xfM/3feN5hoY0gnqcsz15bvM8y7Pc5llurbM9ThAEQYYEYAwNDQgC6oH8IAi9gMtXrzp1aPT5p5/0fD/LUkEk0UcnJ4fTaAwAmj7fD04LC2EYFvwg8D3f84yhwRnqcc7l1to8z20uwQ/8crlcKpUmJqfWb34hnDv/jvvvP2fxZRJ66JnjBw+t/vsfTe567f3XXjtnzuw4irqdbpamgvyeIPB833ieoSEgQZDrs1me51mWpT1JnKR5luZ5bm3uHAARfSQ9YwCQVA/k+17gh1y+etWpQ4eeX/5UWAiTLJUcQEwT5JwgsMfQGM94XhiEYbFQKJaKhUJYKARB4Pu+oaEhBEiCSGKak7NvARAEQRgWnHNbX9q+f/zEDZ/93NV33hGGAUlr7Y7nntvwj/9w7kDtmquvKhYKaZpmWSbJGM/3PM/36REghHeS4JztybI0TdIojuI4TpI4TRLrrJyD0EPSGAIgKAiA53mBH/DpNatPHhx9fvkThWIxSRMnh3dwzkGkMZ7n+b4fhIViqVgql4vFUqFQ8APf0GR53mm3G81WHMdZljm5wPeLhWKtVq0N1MIwBCDrBNHQ0DPk/gMHtuwYnn/Ndbd98Usz5s4hOXls/Jm//dsDm9a//5prFl2y0DMmd1bWgTQ0ALIsa3XarVYrihNrc5CBH5SKxVqtVq1WPc+TlOdZmqRRHEXdThR10zTNs1zOSgJIwpDCGZ7nhX7A5WvXnDp4aPXyx0vFUpzETg7vIAmgZ/wgDArFYrFULlcqpXIpCEIAURSNjx8/OjZ2amKi2WqlWeacAJEM/KBSLg8NDZ4zf+68uXMHajU/CAwJQE6Tk5NbXtzW8MMP3f/Akhuud5ndtWnTM3/z18Uk+uAHbpo1axYNJcg5m9t2u33s+PGxY+OTjal2p5tmmZUD6BlTCMNqpTxzaGj+vHlz5syulMok0jSLo26n2+52oySOsjTVNBKggSSAQOAHnudx+do1E6Ojq596vFQqRXEs59BDSEIfjTFBEBaLxXK1Wq5USuWy5/mtVmts7NjRsWMTjQY8vzZYr9QGwnLJ+IEFkiyLo6jTbHYbU8yzGdXKOXPnzp8/f6heD3zfOhdH0a7de17Z/8bi2++89fe/4LJ87c9++sqzTy+95OIrli4tlkokszxvNJpjx8bGj59odSNTKFQHhwqVqg0C53kS8ixN4zhpNaOpqRCaNVA7Z97c+fPmVStV52wURd1OTzuKunmWOzkCIuUcAQGFMPTocfnaNZOjo6uffrJYKMRJ4qylMZIgAaSh7welcqlcrlRrA8VyyfP88fHju/fsOXbyZKk6cNGlly5asuS8BQtqQ0MslnLjpU5xnkXdqDU1OT46Orp3ZGzfSHzy5Ixq5ZIFC86ZNzcMgjzPx48ff3HHzrw+9OGvPuD73uq//1s3ceLa97539uzZvu+naXps/PiBgwcnW+3B2bMWLVl68WWLZ553vqtUUz9wxjjJZnkSddsTE+OHDh4Z2TO+f59tTs2bOeOyRZfOnTObZBRFnXar3W5H3Si3mSAIkgg4pzAMfONz+bq1E6OHnl/+ZLFQiNNUTiQkASTg+X6hWKhWa9VarVypAjh0+PCOnS934mTJVVdfd/Mti6+4Ymj2bBuG7dx1nU2tywVBAOSUp2lrcuLw3pFdmzft27Y1zLLFCxecf+65YRh2u91de0Z2HxpdeMNN5Wp1/5aNC+bOvuySSwqFQpbnh48c3ffGAVMqvff666+58f2LliwtDQ02YSazLHGC8DY5l8Zx4+TxQ7te37V588Ed22eUi1cuW7bgoguds1EUNZvNdqudJFFuLSQ5gZAUBgXfM1y+bu3k6KHnn36yUCjGSUr0SAB7wEIhLFeqtXq9Wq3SeIePHFm3cWM3s3fcc++9n/vceRdcYMGpNDuZpJ3cOTgQAAlAUA9kYHoaJ0+8tHLFlicfR7Ox+OIF58yb6xnv1KmJbTtfnkjSUrk8VAyXXbpoxtBQbu3R8fG9bxwo1Qc/9pn7PvqJT8yaPTvO7ck4HY+TRI4kAAICIAmAZIwBOHlsbNNjj7349BNDYfiBG66bN28upHar3Ww22+12lqXOOmtzGEIohAVjyOXr1kwcGl2/ckUQBFEcAyQAwvN8z/OKpWKtNjBQrwdhOHZsfNOWrZ0su+u+z9716U/PnD07l6bS7GSSda0VAAHEu6gHgIzxkqg7vOb5jY/+Ijl+bME58+bOmmXINw8d3vn6LkmXL7ns4osuBDB+4tSB0cP1OXM+8ftfuPUjH63V69a6dm7HoqSZ5+ghiDOEaRKmkWw3GlueeHzLrx+tOnvzTe+fP29unuXtZqvRbMTdKM1S6ywAyZUKZUB8et3aU4cObly10vP9KI4AejQAPM8Li4VyuVofrFcqlW63u2b9xuNTjfvuf+COe+8drA86aSrLjkVpN89BQABxlgBIgAQCkkCmcfTK+nVrHv5pdPTwxeedO3fWrDTL9r95MLd24YKLyqXiyYmJQ0ePzTr3vE9+8Yu33H5HqVxxUup0IklOxGkugQBEEO8gTBMEQeo0m1ue/PWqv/u7BfPnfvADNw3W63EUTU1NtdutOEryPHOQnCsVSqS4Yt3ak6OHNq1a6QdhFHUBEjDGBEFYLBVr9T7P87dt3/7CSzvu+r3Pfe7++wdnzCTZsfZwN2nnOSSQgPBOkgAIPSIgACKYxNEr69au/ocfd8aOXHLhBXNmzsyy1DpXKBQmpxoHRo/MPPfczz/44C133BGERfUQjTQ/2o271qKHIN5FAAUQAiD0SGqeOrHqJz/Z+tgvb3jf1ddf8z4CzVaz2Wh2Op0sTa3NBRSCAimu2rxp/I03Nj//nBcEaZa63BljPM8rFIuVSqVWr1cq1VMTEz99+OcXLl78R//uTxcsXBQEvgWORcl4kuYS8W4SpglnCSKmkR69F1csX/4332e7uXjhghmDg5BOTU7tfuNAsT74xa9//c57P1EoFEECiK09Ficn00wCIIKCCOJ3kEAjOcBZe/Lw6MP/4d83D71536c+MXvmzDiKGs1mu9mM4zjLcudsEAS+53P9S9uOjoxsWbva88M0jeVEY3w/KBaLA/V6tTYQhOHKVau2vPTSH/+f/9ed99xbLJVIdnI7GsWd3DoIIDFNwlsECO8kgYAgkUyjaPvKZ1f/40/c1ORlFy8AsPfAgbBWu++rX73rU5+u1OoOIJA5dzJOx+MkhwgIBED8fxAACc7lncmJHatXP/s3P7j+ystvuen9WZq2Ws1GoxF3O0mSOeeCIAjDAtdve/HoyN4ta1f7QZAmsXMwngkLhXKlMlAfrFSqjWbz4Ud+Mfv88//1f/yzCxYspKGAU0k6FiepkwAKgNAjCQDRIxB9Aii8TQApEeg2m9tWPL3qH34cT5woFApDc+fd99Wv3nnPvZVaTYJISc0sH4uSTp6DxDQCkIQ+AiDxDgIISZBcNNU4enD/0z/8m+7Bg1/+/O+Vy+V2q9WYmmq3m0mcWuuCwA/DItdv3Xp0797N6573gyBPE2etFwSFYrFSrdbqM8rl8iuvvfbYE09890/+3d33fbZUKhOw0tE4OZmk1kEQAQGE5ARjAMmJhEAQ0yicJpwmUEi6nW0rlj/7o78t+d4XHnrozk98olIbAA1BQYnT8Tg5maQOIiCBhARINAQoZ2kMQEAQQGKa+lzcbk2dOLF91XNrfvKjT370o5cvXRJF3anJqVazkURRbq3vB35Y4NotW8ZG9m7ZsMYPgixN5JwfhMVSaWBwoDYwZDxv5arVB48d+/ff+8tFi5fQ0AiJ3GiUNNLcyaFPAAxgsyxqtZ1coVzxCwWQIggKAoizhB6hp9tsHdi2dQD2A7fcPGv2XJCY5oCpNDsWJbGz6KMgApDyJEnaHZDFas0LAxAABRBnCIJzUafdnpwc3bP7kf/855eeO//T996TZWlzqi/qdrMsD/oKfH7TpqMje7duWOsHQZYmksJCWCpX6oOD1fpgnKS/fPRX8xYu/Jf/4T/OnD2bAoHYuUPduJnlkgAB8GhOvHlg++rn9r/8SprGcy5c8N4P3XbR0mVhqQTiLcRZAiBBQJCl8wrhnHrNGAMQgKROno/HaSPLHPqIPpumh3fv2rF61ZF9e43nX7hk2dUfvn3exReLPQCIaepzUbvdbTYmjo394j//ORpTD33tK77vNRuNxtRUp9PJ0sz3vSAocvXGjUdH9m7duNb3gyxLIISFQrlSrQ8N1uqDk1ONn/38kVvuvvuhf/7H1WpVAIEot6PdqJVbAZKMYfPY+PL/8YN927ZWgyAMglOt1tBFF9983+9ddu21YbEIAaTQR5wmABJ6Sp4/txgOhr4hBaTWtfO8meZta3M5CSAI5mnyxvaX1jz8s4mDBwbKZSfXyuwl115/+5e+MuPcc5wTSQCSAMi5bqsZtVudZuOJv/je0Z3D3/r6g7Vatd1sTk1OddrtNE08zw+CAldt3HhkZOTFjesCP0jThEChUChXa/WhwVp9aPz4+MOPPPqZ++//vfsfKJZKIChEuRvtdls2lyjJANuWP/nUD38ws1hYvHBhuVweP3Fi1xtvDFy04CMPfvPiy68ghB7iLUSf0EcDljyv5BmfdEDiXGxtJuec0ENAyNP0jeEdz/3kR6fe2PeehQvPnT8/TdL9Bw8ej+I77n/wunvuydOcJHoEQc7mnUYjjrpZFK388Y9eefaZbzxw/5zZM9utVmNyqt1up0lCY8KwyOc2bjwyMrJt47rAD7IsJRAWC+VKdXBoxkB98PDY0Ud++egXvv0Hn/rCF8JiEYABEqtD3W4jywVIcnn25F9+76XlT75vyZJLFl5cLJayLNu9d+S1ffvmLr3iA/d9bsGyZWEhhABiGvEWoc+AJAwowEkCBBGnKU+Sg6+9tv6RfxrduWPppYsWX3ppMQzjOH5zdHTnyN4r7vzYJ//on6dJSoMeCYDyLGtNTmRZZtNk7cP/tPXRnz94/9fOnT+/025NTUy2260kSTzPC/wiV27ceHRkZNvGdYEf5nkCICwUK9VqfXBooD54+OjRnz/66O9/+7uf/sIXC8UiAANkTke60USaWcgJyvPlf/1XWx9/7Or3LL7s0ksr1Yrvea12+7Vdu17f/8Y5V1x102fuW7B0WVgogHgL8Q7CWUSfAAIEbJaN7t616dFfHn55x8XnnXvF0qWVSiXP8263s//Amy/v3XfVXffc850/SNOEJAD1IYuj5tSknMuTZO3DD2999JGHHvzaOfPnd1rNyYnJTrudJInneYFf4MqNm46OjLy0ca0fhDZLAfhhWKlW64NDA/X6+IkTP/35I5/62v2//8BDxVIJAAEnHY+T8STNnJycAV9+fvWvv/ff6p659uqr5s+bXyiEIBvN5quvv7539Mj8y6+85b7PXvCeJZ7voY84g/ifEkCXZYdHdm969BejwzsuPu+cZUuXDNbrcsqy7PiJ49uGdzasvesb37nyw7fneU4SgHqcizrtdqtFKYuilT/6u1eee+abDz04a+aMdqvVmJxst1tpkvo9XsiVGzcdHRnZtnFtGIR5nkHyg6BcqQwMDtUH61PN1k8f/vlNH7vr63/8L6rVKgFIAJp5frgbd61zkiE7kxPP/v2PXn1+1fmzZ125bOmcWbP8IADQbLWGX3ll5NDogutuvPmznz3n4oXGGIA4g/jdBMHl+dj+vese+aeRzRsXX3TRlVdcPmNoCESe5acmJna++uobR8euvO32D3/5q9VZsyQQEAQgz7JOsxHHsQG6zcaT3/vekVeGv/X1h6rlcrvVnJqc7LTbWZb5PV7IlRs3je0d2bZhXRCGeZZKCoKgWCoNDA4O1AfTLH/017+ece75/+rP/mzOnLkQCAnIpKNxMpGkuQTAGHP8wIFV//iP+7e9sPCceUsXL54xNBQEvoDJyakXt28/MHbsPbfe9v57Pzn3wos8PwAhgCDeTegRBWftyYNvbnr8Vy8/9+w5s2bedMMNM2fMIJnl2eTk1Kuvv777zYML3nftHV/8yvzLLpNEQgIkwcWdbqfdcrk11KmxsV/++X/yOu0H7v+KAVrNxtTkVLfTyfPc7/FCPrdx09GRvds3r/eCIM8SCJ7vF4rFaq1WHxz0/fD5detGDhz80//6X5dd+V7f8wRAEtDK86PdpG2tgyiQOLxnz5qfP3xw+7aF58xf+p73zJgx5Hmete7EiRMv7dgxevLUVR+9+wOfvm/GvHNEgiB+mySQbJ88+cITj720/Mk5A9WrrrhizpzZxhhn3eTU1Ou79+w5eOicyy+/44tfOe/SxfAM+iQBUJ6lnWYziRNCzuaHXn/9V//l/1l28YK7P/aRJI4bjUZzqhlHnTy3QRD4XsjnNm0aG9m3ffN6z/fzLHVynueHhbBcqdYHB8uV6u49I79+8qnf//Z3PvvlL9eqAyIICLDSZJqNx2lkcwEQ4Nyh3btW/+ynozu3X3bhBcve857BwTrJLMuPjY+/8OK2rFK9/WsPLHn/zV4QgCSEdyEgCR7NgZ07ln//L7KT49dfc838efN83wfRarVf371n95sHz1l2+Ye/8KULlixxTiB6BECQbNTpdFot52Rkkzje8cwzm3/5yKfv/tiiSy7ptFuNqUa71UqiyDobhgXPBFy1afPY3r3bN28wvp+niZMzxvPDoFQsD9Tr1Vqt041+/stHC4ODf/Lnf37JokthDEEAApx0Ik6OJ2nqnAAIhji8d9/yv/n+2GuvLLt00eJFiyrlsnU2TZJde0ZGjhy99lOfveHeTxTLFZIgAaGPeIsgTxhe+eyzP/yr82fNuHzJklK5bIyJ43hk//6XXn39nGWX3/XQN86/dLEAkBJISAKQZ2mrMZUmKSE4OzV+4pkfft+ePPGlz3/OI5utZmOq0e10sjR1zhYKRUOfqzZtPrZv/0ub13uel6WJIND4vl8sFsqV6sDAYLFUWrN+3YbNL3z73/7bj33ik5XaAEkATjJAZN2RKGnkmXMSQcnzgzd27ljxt//j5L6Ryy9ddNH55/u+l2XZyN59b56cuO7Tn73u7o8XyxWSIN5CQABxmrRz9cpnf/BX8wdqSxZfViwWsix/89Dozl27a+dfeNeDX7/suusEgCTeItk8j7rdbqclBwPlWbZr48Y1P/nRB6593w3XXdvtdFutZqvZjKMoz3NnbaFQNMbn6s0vjO3b+9Km9Z7nZ1ksgYbGeEEQFIrFer1eqw00W62HH/nFzPPO/9a//jfvWbrMDwKQBBwg6XicHE/S1DkAEigJGnlp28of/2hy/76F5583f+6cTrf76q49wdx5H33gG4tvuNH4HvE24p0EY3jw1Zef+selzMAAABPoSURBVP5fTh3Yv/SyRdVqdfz4iZE336xfcNFdD31z0fuuASGS6JMAydo86rSjbhfOEYDUPHHiyb/87+mJ4/d98hOD9Xqz1Ww1Gt1uN00S65yztlAoesbn8y9sPbp370ub1hnjZWlMEsaQ9DwvCMJa30CpXH5t1+5nVz9/y113f+0735kxcxaMMaQDCDTT7EgUd60V+gQSsnm+f8f2FT/8wZsv7ywGQZImpcEZH/rCl6+/597q0BDOICCAAAFhmgQScav50jMrVv3j37dPHC8EYerchZe/986vPXDJ+64GCUEACUASbJ5H3U7c6UgyEIROo7Hu4Z+9umb1nbfe+t4rL8+yrNlotNutOI7zPJdz1rpisegZn2te2Dq2b9+LG9d6np+liZPzfA+CMcbz/FKpVK3VKpWq8fwVz658Zdeub/7Lf3PPZ+8Li0VJRF/s3OFuMpVlggRSAik5QzNx+PC2Z1eM7HhpYMaMaz5y98Krrw6KRQggiB7iLOIMYVrS7b4xvP3Fp59qnDhxyVXvu/ajd82+8EIra2gEEQTU45zttlrdbpeCgZNgs/zFJx9/8gffv/Haaz5+18cItdqt5lQjTuIszZy1kpxzYRB6XsA1W18c27f3xfVr/aCQZ4l11vgGTjQGDmEhrFSq1VqtXC5PTjVWrV176OjY5x/6+t333Tc0Y6YxBlImHI3ik0lqIQMIPQSEHsHmeZ5m8EwQhsYYnEX0EQAJCT2kJLxFzro8TWWdFwZe4JPEO8nZPI+6nW63K+cMZISo3R5eteqFJ349tz7wsTtvHxoairqdZqPZ7bTTLM3zHILkAHq+75uAa1/cNrZ3ZOv6dUFYyNLYutzzfTkHUE6+74VhoVKrVivVQrE4fuLk6rVrm3Fy7xe+eNcnPzlj5iwaI+B4nB6Lk0xOAEFAADFNOIN4J+Is4l2EPgKCAAiE0EecIQGQy7O42426XWutgQikne7La9dsefyx+YMDt978gZkzZ2Zp0mq1Oq1WHMfWWZ0GwMkPAt8EXP/S9qN79mzdsNYPClmWWJt7ngdAfY6C5wfFYrFcqVSrVT8IT01NbnrhhRON1qe//JW7P/OZarUmaSrLD0dxbK1IAoTQRwl9BIQeEm8hziD+JyShTwSEHqKHggDR2SzutJOom1lLyJBJt7v92We3PvnEebNm3HTD9fPmzs3zrNNqtVutbtTNs8xJOI2Uc74f+Mbn+u07ju7Zs3XDGt8vZFlibe55niAIzlpJnvH8wC8US5VqpVKphoXC+PHja9dvjKz78ne/e/OHby9Xyu3MHupGHWtBQgBEUgJAEhIA4TQS70D8DgIgAsI0EhJ6CAmAZHObdrtx1LbWCqKURPHuzZtX//QfZpaLt9/2oXPmzbPWdtrtdqsVRd0sTZwEgKAgSAJ8P/CNzw3bh4/s2b11wxrfL2RZYp31PCMnQC63kgAaY4IwLJaK5Uq1Uq36vn/w0Oi6jZv8au1L3/rWjbfcgmLxUDfu5FYg+oQ+AkSfcAbRR7yNQo9wBtEnAiIhERDepj6XZ0nUTaLI5bkAEHmSvDE8vOGXj3jdzq0fuOmCC86Xc51OuyfqdrM0yXMLgIY4TQLpe55nAm7YMXxkz+6t69f4QSHPEuusMZ7kINg8h+ScaOh7vh8ExVKpXK1UqlWAe/fv37Rl67kLL/n8gw8suurq4w6t3AJEDwUQZxAQ+og+4h0I4d2EPoKCAEIAAQmQnMuzNI2iLI6stRJIOevG9u3d9OgvO2NHbrjmfZddcomTup1Ou92Kut00SazNrXOGxngGpwmCfN/36HPjjuEje3a/sH6NHxTyLHHOGuOpx8k5K9fHHtPjBWFQLJWq1Wq5UnHCnpGRLS9tv+KGGz/1wENm3vyuk0ACoADiXYgzCEDoI3pEvIvQQ0BCDykIPZKzeZrE3U6eJrJOAARQk8eOrXv4Z8d277ru6iuXvWdJ4HudbqfdbHU6nSxN8ix3EgAangZAEqDADw09bhwePrJ7zwvrny+ExSRNnKyhUY+bZi3eRnqe54dhuVyu1mrlSsU5t2HzloNHxj74mfuu/OjHg3rdoY8UQJxFnEWhjwAh9BFnCX0UBEAg1QNn8yyN07ibJYkkCD2Sus3G9udWbn3sV1csvuzmm24sFYtRFLWazU6nE0extblzDtNo2GOMkQQJgO+HhoYbh4eP7N69df3aYrEUxZFz1hgj5yRZa+UcpgkQRNDzvLBYrFQq1YGBSrlyanJizboNXePd8eA3L73uej8MhR6RxFnENIGYRoAQ+oizhLcJPQKck8uzNOlmcZxnGSAJ05TG8f4dw+v+6WcVuY/c8eG5c2bHUdRqtTqtVhzHWZY55/AWGvYYUpgm+L5PetwwPHx09+4X1q0tlUpRHDlnjTFyTpLNcwGQAEhyEgCSfhAUiqWBgVp1YCAIgpG9+1av33DBtTfc8dX75110EWgAkJgmgAABCMQ0AoQwjSAg9FEQeoQ+Ak7W2ixL8yTO0tjmFhIBAepzE8eObfjFL8de3fnhW25edMlCm2etVrPdakXdKMsyay16CAg9NDTG4C0SAs83ns8Nw8NHdu/Zum5NqViK09g5S8I59bg8B6AeAJJzTuihMQwKhUqlMlCvV6rVNM3Wbdi459Dh2756/3Ufu6tYq0kiCGIaMU3oIQFCmEb8FkHoUY/NsjSJsyS2WeYgSBAoiJCUpcneF1/c+POHL54/77ZbP0ii0+k0p6Y67Xaapm4a+yABpCFpSEDooaTQD4zxuX54eGzPnhfXry0UinESSw6Sc049zklyEgEnpz70kX7gF4ul2kBPvVAsjo2NPb58xczFSz7+jW/Pu2QhaUiC6BNF9BA9BEScRRCQ8BZBknM2z9Is7mZx4pwlAFIShB5BIJsnTqz7p4ePv/7qR2//8PnnnxfHcavZbDYaUbeT51aAnOM0AAKMMZwmiaAg3ws843H9zuHxkb3b1q/1/SBOE8nJOU0D4JyVIMk6B0noI+l5XqFYqtYq9fpQqVzO8+yZlatGG82PffPbS9//gaBQBAEQgACoj+oBoB5IEARB6BFOE5ycy7M0yZLY2RwgABoSRI8AQoCkg6+8svyv/+qiObM+dMstlUo5ieOpqalWsxnHcZ5nACEBIAkSgCHBPkASJBWCAmi4fufw+J692zas9f0wTiLJ9YCUc+pxzkkArBycBIAwxgRBUCiUarVqrV4vFIrO2dd37960bcc1n/z0tR+/t1wbsDZ3eZ5neZ5leZLmaZJnSZ7lNsvyPHO5dc7JOTk5iBJJQM7mzlpJxtDzA8/3vTD0wyAIQs/3jef5QeAFQZ4mL614ZuPPH7715puuvHxZISwkadJsNluNRhxHeZa7HsmQBAWAMCRIYzw5JwhA4Aek4Yadw2O7R3Zs2mC8IE661lpAJCU4ZwU45yQHUM4JMJ7xjAnDQqlSqdUGKtWK53lJko6PH1u1dv3QosuW3vxBGq8xcbJ54kRrcjJqtaJWs9tuRa12nqaaBkkAJLyLJAACwD7jh2GhXCpWKuVarViplAcGBmbMHJgx0xjz+qZNR1/d+ZHbbltw8UWlYsk51+m0m81mt9vJ0tTmubWOJEgABEAShoYA1OfCsECAG3YOH9295+UtL9CYbtQhIAiSAOesJOesBAGCIHm+H4ZhqVSu1qqVai0MwizPT5w48dquXduHd56cmAQJgYae59Wq1frQ0OCMmUMzZtQGB8vVaqFUDMNCWCj4QeD1+L4xHiFrnbW5tXme5VmapEkSR3Gn1Wo1G82pqclTpxpTU+1WK8sy5xzUVx+oXfe+qy9ftnTOnDlhEGZZ2m61W61mHEVZmlgr5xyJM0hDQ2MkAA5gGISAuH7n8LHdIzu3bAEZxV3JQSAh9FmbOfXAyRGgMb7vl0rlaq1WrdVKpdJUo/nyK6/ufPnlKM/PufCiCxYsmH/eefPPO2/WnLmDM2YMDg5VatVCWPB83/M8egbsAQSQAAgI7yZhmpxzkrPW5jZNkk67PTU1OTVx6uTx48cOHx47fHj0wIHDBw8G5FVXXn7FsmVDg4NxknQ6rWajmcRRluZODtNIAjDGg9AjOAEFv1DwQ27YOXxs997hzZtp2I07kggIIOmcszYTIMA5R9IYUygWq9VabWCgUqlGcfzc6ucPHBpdds01H7zzI5cuWTI0a3axVPJ8z9AYEsaQAIgegQQgEGcI70K8TQABgoIoCJAgOSfJudzmSRRPnjy5+7XX1qxYsWPLpkUXXfShWz84WB/odrvNRqPdaiVJ6pyVRFISSWM8AJIASCoVSoHnc8PO4WO79w5v2QQwTrpOgkASgJV10wAIIukHYalYHKjXawN1z/NWrV376uu7P3bfZ+/6zGfmnXNuEIbG84wxAEicJbxNANFHoo+Q8NsIQAABgYQAAQIISiLgnLPO5ml68vjxZ5968rGf/MPSRRd/6IO3SGi1ms1GI+p2sjyXE6YZ4xGkobOOhk6u4BcKQYEbhoeP7RnZsXkzySjuSg4kQQcHIbe5JBCSjDFhWKhUq/V6vVKtjR4+/Mijv7rihvc/+Ef/7OJFizzfB4nfhYDQR0DoI95C/CYBBIW3CWcIv8W5KM9fe/PNh3/w1we3bL73ro+ed+45rVar0277QTBj1pxKtQqg1WodHzs6NTHprHXOkpRUCIuhH3L98PCx3SPDW7YAipMIkEAA6oGctYKcHAljvGKxVBsYGKjXg7CwfuPGbTt2fud//5Pb7767XK4IEECcQfQIIAACAog+SSBxlkCiRxD6iB4BJHoEUBJIAIIESnKClcucIms7uT3Zam1dverZH/zVdcuW3HTD9VmWFYrlwRkzg0LoGU9AnuftZvPQgTeOHT3irBUgqVQoBV7ADTuGx3bvGd6yBUCcRZBEygmA5Jyck5ysofF8r1QsD9TrA/V6bt0TTy3v5vZf/d//ackVV1ogsS6TBBEgiB6CwhkEBAICQZwmCQCJHgEQ+ogeAgQgnCZAgCQrOCmTUrncKXMuk9I0O7h71+P/7b8M0X363nvLlcqMWXP8MGg2GuPHxgy9OfPmhoXixKmTr7y0LYq6BJ1UKpQDz+OGHTuO7tqz84UtBOM8ds4CdM5hmpMT1APA8/1yuTI4OFgbqLc7nV/86rFzFi78wz/9P2add/5Emnaty52EtxEEhR4BICj0kZgm/Bahj+ghQAACIABCjwQJBOAkB0gQJEBOJw+PPvEX/725d8/Xvvqlc889f2jW7G6ns/u1V4+MHvQ8f+78+ZctvTxJkuEXXzh1/DgAAaWwGPoB1+/YPrZrz/ALLxgyyRLnrADrHCCCgv7f4uDlN66zAOPw7/3OOXPxeI4npiAhqMIKtaiNEGWBoPuqVMCuG6TSlKhdIfXfQUi0y0pRUlgAKpdFE7eS3XGEBcRxAsRW3DhRPHZsZ67nnO9lxk0IhSJllT5PtAE7pmk615rvdDrz+cLdg4Oz585/8/nnf/yzN7V4Yn8yKQ0EwOb/E49M/CebKUsCxJQAA7Zg//at3/7i59c//OCNM6e//tTT7Xyh3+/vbN+4d3SkEOqNxpe/8tWqLC//Ze3G9U0wUiOrZ2mqi5cu3VzfWFtZDtKoGDnGCHa0kTA4GmycpGmrNd/pdObzvNfbO3v+3e+88OILPz0T2+2iKlGQghCPkUFG9n7vzntv/fLKn/7wxpnXnnn21FxrfjQaHR0dFpOCB6qq2vz7te2tTUlI9ayeJYkurK7uXLm6trIcpOFk6BgjeAqHEBxtjG1I03Su1VrodNp5vrvbO3vu3e/+8Effe+UnSZ7LVSCgEJiSmRGPxMyIR2VmxIwBY3y41/v922/99Xe/ef2108+eOtVq5/1+f3trs9fbFSBhYqwO9vYO7t6VBNRrjSxJdKHb3blyde2j5aAwGA+AGG0siQfsaDvN0mZzLl9YyBc6d3q9s+fOf/vFl557+eVsvh0cA5pBgAEj/pu4zzxkZsRDYsZ8ikHMGAQCg8GAfLS///4776z/8b3XT7/61NPfWDixOBwMbmxtbv7zH5PxOISQJElVVYN+P1YREWPVrM9lIdWFbndn4+raynJQGE5Gxo42SEQ7SByzY0iSRrPRbuf5Qmdvf//cr369ePJrT37ruaRexxYSD5j/JT7FfDbxkPlsYsYCg5gaD4bXuh8dbF0/8+orT548ufjEl0IIg0H/zq3bR4eHIahWq9+7d7S9teVoiJVjo9bMFHSx2725cXVtZUXSpJhUjjZgScZCHDOEoHq93mrNz+ftwWD4/tLS7Tu7lgTGfJ6EXZXlF050fvDS9/N23mq384VOSFMdc4xFWd7euXntb5ersrSjRZpmtZBoqbu6vbGxtrycJsmoGEcbsA1Isi0JEKCQZWmj2ZxrtdIsOzg8nBQFnuFzYyNAYKhltS8+sVhMiqIsm81m3jnRaDRCklRVNRoO9nZ7tz7+2DEaT6VpVguJllZXt9evrK0sZ0k6LMZVjJJsB0khOEajoBnjEJIsyxrNxtxcq9FspFkWgqQghI2wxAMCYyHAtiQMMmDuE/8mpmwkZmwEBgmDmDISNpaFjDFgkKNjrMqyHI/Hg/5gNB4V44lAEmIqVjHaVVkiPlHPakHS0uql7cuX/7yynCbppCwMVaxCCGJGyNynmZCkyVStXptK0zQkSVCQQIBAPC7GYKZMjI6xKouymEzKoiyKIlZlFSssEyXZGGQkGU/Vsgyki93VG+vrqx9+UE8zhTCpimgLSYAkQFO2wZLSkCIpUZIk4ZhC4BMSj5nNMUdPxVhNxSpWcQZjogxSjFEhBKmKlVCWpFmaDCeTfwGr+BN73gl9XAAAAABJRU5ErkJggg=='

In [5]:
img = base64.b64decode(encoded_text)

In [6]:
with open('testimg.jpg', 'wb') as f:
    f.write(img)

In [ ]:
def decode_image(imagestring, filename):
    img = base64.b64decode(imagestring)
    
    with open(filename, 'wb') as f:
        f.write(img)
        f.close()